<a href="https://colab.research.google.com/github/AyazMurtazin/DeepLearnPythonKPFU/blob/main/ML/%D0%9F%D0%BE%D0%BB%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BB%D0%B0%D0%BD_%D0%BF%D1%80%D0%B5%D0%B4%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B4%D0%BB%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### **Полный план предобработки данных для регрессионных задач**


DATA SOURCE https://www.kaggle.com/datasets/stealthtechnologies/regression-dataset-for-household-income-analysis

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
# Install dependencies as needed:
# !pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "stealthtechnologies/regression-dataset-for-household-income-analysis",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

#### **1. Первичный анализ данных**


- Проверить размер данных (`df.shape`).


In [ ]:
df.shape

- Определить типы признаков (`df.dtypes`).


In [ ]:
df.dtypes

- Проверка на дубликаты (`df.duplicated().sum()`).


In [ ]:
df.duplicated().sum()

- Анализ уникальных значений для категориальных признаков (`df[column].nunique()`).


In [ ]:
for col in df.columns:
  print(col, '\t' ,df[col].nunique() , '\t', df[col].unique())

- Анализ целевой переменной:


  - Распределение (гистограмма, `sns.histplot` или `sns.kdeplot`).


In [ ]:
import matplotlib.pyplot as plt
n_cols = 2
n_rows = df.shape[1] // n_cols + (1 if (df.shape[1] % n_cols != 0) else 0)

fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(10, 10), layout = 'constrained')
for i, column in enumerate(df):
    sns.histplot(df[column], ax=axes[i // n_cols, i % n_cols]).set_title(column)

  - Выбросы (боксплот, `sns.boxplot`).


In [ ]:
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5, 15), layout = 'constrained')
for i, column in enumerate(df):
    sns.boxplot(df[column], ax=axes[i // n_cols, i % n_cols]).set_title(column)

  - Визуализация взаимосвязи между признаками и целевой переменной (`sns.pairplot`, `sns.scatterplot`).


In [ ]:
sns.pairplot(data = df, x_vars = df.columns, y_vars = ['Income'], )

In [ ]:

fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5, 15), layout = 'constrained')
for i, column in enumerate(df):
    sns.scatterplot(data = df, x = column, y = 'Income',ax=axes[i // n_cols, i % n_cols], marker="+").set_title(column)

- **Корреляционный анализ**:
  - Визуализация матрицы корреляции между числовыми признаками (`sns.heatmap(df.corr(), annot=True)`).
  

In [ ]:
sns.heatmap(df.select_dtypes(include='number').corr(), annot=True,)

-
  - Проверка мультиколлинеарности с помощью VIF (Variance Inflation Factor).


In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

data_with_const = add_constant(df.select_dtypes(include='number'))
vif = pd.DataFrame()
vif["Variable"] = data_with_const.columns
vif["VIF"] = [variance_inflation_factor(data_with_const.values, i) for i in range(data_with_const.shape[1])]

print(vif)

- Проверить дисбаланс данных.



---



#### **2. Обработка пропущенных значений**


-
  - Анализ количества пропусков (`df.isnull().sum()`).


In [ ]:
df.isnull().sum()

-
  - Разделение на типы пропусков: случайные или систематические (например, через визуализацию).


-
  - Для **числовых признаков**:


-
  -
    - Заполнение средним/медианным значением.


-
  -
    - Интерполяция для временных рядов.


-
  -
    - Использование KNNImputer из `sklearn.impute`.


-
  - Для **категориальных признаков**:


-
  -
    - Заполнение модой.


-
  -
    - Создание категории "Unknown".


-
  -
    - Frequency Encoding (кодирование частотой встречаемости категории).


-
  - Удаление столбцов с >50% пропусков.


-
  - Удаление строк с пропусками (если мало).


-
  - Создание индикаторных признаков для обозначения пропусков (например, `df['is_missing'] = df['column'].isnull().astype(int)`).

---



#### **3. Обработка выбросов**


- Визуальный анализ (`sns.boxplot`, `sns.scatterplot`).


In [ ]:
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5, 15), layout = 'constrained')
for i, column in enumerate(df):
    sns.boxplot(df[column], ax=axes[i // n_cols, i % n_cols]).set_title(column)

- Методы обработки:


-
  - **IQR-метод**: удаление значений за пределами [Q1 - 1.5*IQR; Q3 + 1.5*IQR].


-
  - **Z-score**: удаление значений с |Z| > 3.


-
  - Логарифмирование для право-скошенных данных.


-
  - Квантильное преобразование (`sklearn.preprocessing.QuantileTransformer`).


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import QuantileTransformer

# Создаем синтетические данные с выбросами (пример)
np.random.seed(42)
data = df.select_dtypes(include='number').copy()

# ---------------------------------------------------------
# 1. Обработка IQR-методом
# ---------------------------------------------------------
def remove_outliers_iqr(df):
    df_clean = df.copy()
    for col in df.columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

data_iqr = remove_outliers_iqr(data)

# ---------------------------------------------------------
# 2. Обработка Z-score
# ---------------------------------------------------------
def remove_outliers_zscore(df, threshold=3):
    df_clean = df.copy()
    for col in df.columns:
        z_scores = np.abs(stats.zscore(df_clean[col]))
        df_clean = df_clean[z_scores < threshold]
    return df_clean

data_zscore = remove_outliers_zscore(data)

# ---------------------------------------------------------
# 3. Логарифмирование (для право-скошенных данных)
# ---------------------------------------------------------
data_log = data.copy()
# Применяем только к положительным признакам с правосторонним смещением
for col in data.columns:
    if data[col].min() > 0:  # Проверка на положительные значения
        data_log[col] = np.log1p(data[col])  # log(1 + x) для избежания нулей
    else:
        print(f"Столбец {col} не может быть логарифмирован (есть отрицательные значения)")

# ---------------------------------------------------------
# 4. Квантильное преобразование
# ---------------------------------------------------------
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
data_quantile = pd.DataFrame(quantile_transformer.fit_transform(data), columns=data.columns)

# ---------------------------------------------------------
# Проверка результатов
# ---------------------------------------------------------
print("Исходные данные:\n", data.describe())
print("\nПосле IQR-очистки:\n", data_iqr.describe())
print("\nПосле Z-score очистки:\n", data_zscore.describe())
print("\nПосле логарифмирования:\n", data_log.describe())
print("\nПосле квантильного преобразования:\n", data_quantile.describe())

- Учёт контекста задачи: иногда выбросы могут быть важными сигналами.
- Использование методов, устойчивых к выбросам (например, модели на основе деревьев решений).
- Проверка влияния выбросов на качество модели (например, сравнение метрик с удалением и без удаления выбросов).

---



#### **4. Преобразование категориальных признаков**
- **Label Encoding** для порядковых данных.
- **One-Hot Encoding** для номинальных данных (с `drop_first=True` для уменьшения мультиколлинеарности).
- **Target Encoding** для категорий с большим количеством уникальных значений.
- Учёт количества уникальных значений: объединение редких категорий в одну ("Other").
- Frequency Encoding (кодирование частотой встречаемости категории).

---



#### **5. Обработка числовых признаков**
- **Масштабирование**:
  - `StandardScaler` (стандартизация).
  - `MinMaxScaler` (нормализация).
  - `RobustScaler` (при наличии выбросов).
- **Преобразование распределения**:
  - Логарифмирование.
  - Квантильное преобразование.
  - Преобразование Бокса-Кокса.
- **Статистические тесты**:
  - Проверка на нормальность распределения (например, тест Шапиро-Уилка или Колмогорова-Смирнова).
  - Проверка гипотез о равенстве средних (например, t-тест или U-тест Манна-Уитни для сравнения групп).
- Учёт масштабирования для моделей, чувствительных к единицам измерения (например, SVM, KNN, линейная регрессия).

---



#### **6. Feature Engineering**
- Создание новых признаков:
  - Взаимодействия признаков (умножение, деление, соотношение).
  - Полиномиальные признаки (`sklearn.preprocessing.PolynomialFeatures`).
  - Групповые статистики (среднее, медиана по группам).
- Для временных данных:
  - Извлечение дня, месяца, года.
  - Создание лагов и скользящих статистик.
  - Добавление бинарных флагов (например, выходной/будний день, сезон).
- Генерация бинарных признаков (например, флаги "больше/меньше определённого значения").

---



#### **7. Отбор признаков**
- Удаление коррелирующих признаков (`df.corr()`).
- **Проверка мультиколлинеарности с помощью VIF**:
  - Вычисление VIF для каждого числового признака.
  - Удаление признаков с высоким VIF (>10 обычно считается порогом).
- Отбор с помощью:
  - `SelectKBest` (по ANOVA F-value).
  - Методом рекурсивного исключения (`RFE`).
  - Важности признаков из моделей (Random Forest, XGBoost).
  - SHAP-значений для анализа важности признаков.
- Учёт интерпретируемости модели при выборе признаков.

---



#### **8. Разделение данных**
- `train_test_split` (стратифицированное, если нужно).
- Проверка на утечку данных (data leakage): признаки не должны зависеть от целевой переменной в тестовой выборке.
- Стратификация по нескольким признакам (если применимо).
- Для временных рядов — хронологическое разделение.

---



#### **9. Дополнительные шаги (опционально)**
- **PCA** или **t-SNE** для уменьшения размерности.
- **LDA** (Linear Discriminant Analysis) для регрессии.
- **Балансировка данных** (если целевая переменная неравномерна).
- **Кросс-валидация** для настройки предобработки (`Pipeline` + `GridSearchCV`).
- Настройка гиперпараметров трансформеров (например, степень полиномиальных признаков).

---



#### **10. Финализация**
- Сохранение предобработанных данных (`to_csv`, `to_pickle`).
- Экспорт трансформеров (`pickle`, `joblib`).
- Сохранение конфигурации предобработки (например, параметры трансформеров) для воспроизводимости.
- Проверка совместимости предобработанных данных с моделью (например, отсутствие NaN или Inf).
- Тестирование конвейера на новых данных (например, проверка работы Pipeline на отложенной выборке).

---



### **Дополнительные разделы: Статистический анализ и проверка гипотез**



#### **11. Статистический анализ**
- **Тесты на нормальность**:
  - Тест Шапиро-Уилка (`scipy.stats.shapiro`).
  - Тест Колмогорова-Смирнова (`scipy.stats.kstest`).
  - Визуализация QQ-графика (`statsmodels.api.qqplot`).
- **Тесты на гомоскедастичность** (равенство дисперсий):
  - Тест Бройша-Пагана (`statsmodels.stats.diagnostic.het_breuschpagan`).
  - Тест Уайта (`statsmodels.stats.diagnostic.het_white`).
- **Тесты на линейную зависимость**:
  - Корреляция Пирсона для линейных связей.
  - Корреляция Спирмена для монотонных связей.
  - Корреляция Кендалла для порядковых данных.



#### **12. Проверка гипотез**
- **Гипотеза о равенстве средних**:
  - t-тест для нормально распределённых данных.
  - U-тест Манна-Уитни для непараметрических данных.
- **Гипотеза о равенстве распределений**:
  - Критерий Колмогорова-Смирнова.
  - Критерий хи-квадрат для категориальных данных.
- **Анализ влияния категориальных признаков**:
  - ANOVA (для проверки влияния категориальных признаков на числовую целевую переменную).
  - Критерий Краскела-Уоллиса (непараметрический аналог ANOVA).
